# Марихуана и скорость реакции

Изучалось воздействие марихуаны на скорость реакции. 
В качестве испытуемых
были выбраны по 12 человек из каждой категории:
* никогда не пробовали марихуану;
* иногда употребляют марихуану;
* регулярно употребляют марихуану.

Испытуемые были разделены на две равные группы; половине из них дали
выкурить две сигареты с марихуаной, вторая половина выкурила две обычные
сигареты с запахом и вкусом марихуаны. Сразу после этого все испытуемые прошли
тест на скорость реакции.
Требуется оценить влияние марихуаны на скорость реакции, учитывая фактор
предыдущего опыта употребления.

## Загрузим данные и посмотрим на них

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('./weed.txt', delimiter='\t')
data.head()

In [ ]:
data.describe()

Отделим марихуану от плацебо

In [ ]:
placebo = data[data['Placebo/Marijuana']==1]
placebo.head()

In [ ]:
weed = data[data['Placebo/Marijuana']==2]
weed.head()

Переведем выборку в более адекватное представление: один объект == один человек

In [ ]:
new_data = []
for i in data.values:
    new_data.append({'placebo': i[0]==1, 'time': i[1], 'type':'none'})
    new_data.append({'placebo': i[0]==1, 'time': i[2], 'type':'light'})
    new_data.append({'placebo': i[0]==1, 'time': i[3], 'type':'moderate'})
new_data = pd.DataFrame(new_data)
new_data.head()

In [ ]:
len(new_data)

## Посмотрим распределение времени реакции

In [ ]:
sns.boxplot(data=new_data[new_data['placebo']==True], x='time', y='type' )
_ = plt.title('Placebo')

In [ ]:
sns.boxplot(data=new_data[new_data['placebo']==False], x='time', y='type' )
_ = plt.title('Weed')

## Дисперсионный анализ

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

### Двухфакторный анализ (плацебо + тип курильщика)

In [ ]:
lm = ols('time ~ C(type) + C(placebo) + C(placebo):C(type)',  data=new_data).fit()

---

#### Что произошло?

$$X_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},$$
$$i = 1, ..., K_1,\ j = 1, ..., K_2,\ k = 1, ..., n.$$

- $\mu$ -- общее среднее значение признака,
- $\alpha_i$ -- воздействие уровня $i$ фактора $f_1$,
- $\beta_j$ -- воздействие уровня $j$ фактора $f_2$,
- $\gamma_{ij}$ -- дополнительное воздействие комбинации уровней $i$ и $j$ факторов $f_1$ и $f_2$,
- $\varepsilon_{ijk}$ -- случайные независимые одинаково распределенные ошибки

- ```ols``` -- метод наименьших квадратов для задачи регрессии
- Внутри используется формула из ```patsy``` ([описание синтаксиса формул patsy](https://patsy.readthedocs.io/en/latest/formulas.html#formulas))

Краткий экскурс в формулы ```patsy```.

- Формула разделяется на:
    - левую часть: ```time``` -- целевой признак
    - правую часть: ```C(type) + C(placebo) + C(placebo):C(type)``` -- выражение, состоящее из факторов, чьё воздействие на целевой признак мы исследуем, и некоторых операций
- `~` -- разделение левой и правой части
- `+` -- берем во внимание только сами признаки, не учитывая их внутренние взаимодействия ($A \cup B \backslash A \cap B$)
- `:` -- берем во внимание только внутренние взаимодействия между признаками ($A \cap B$)
- `*` -- берем во внимание как сами признаки, так и их внутренние взаимодействия ($A \cup B$)
- `С(...)` -- категориальный признак

---

Можно посмотреть различную информацию о полученной модели

In [ ]:
lm.summary()

In [ ]:
sm.stats.anova_lm(lm)

Гипотеза о том, что предыдущий опыт употребления не влияет на скорость
реакции, не отклоняется ⇒ данные по группам можно объединить.

In [ ]:
lm = ols('time ~ C(placebo)', data=new_data).fit()

In [ ]:
sm.stats.anova_lm(lm)

### Критерий Стьюдента о равенстве матожиданий

In [ ]:
from statsmodels.stats.weightstats  import ttest_ind

В предположении о равенстве дисперсий

In [ ]:
ttest_ind(new_data[new_data['placebo']==True]['time'], new_data[new_data['placebo']==False]['time'], usevar='pooled')

Без предположения

In [ ]:
ttest_ind(new_data[new_data['placebo']==True]['time'], new_data[new_data['placebo']==False]['time'], usevar='unequal')

Левосторонняя альтернатива

In [ ]:
ttest_ind(new_data[new_data['placebo']==True]['time'], new_data[new_data['placebo']==False]['time'], alternative='smaller', usevar='pooled')

## Итог:

- факт того, принимал ли человек раньше марихуану, не влияет на его скорость реакции
- при выкуривании марихуаны реакция человека значительно зампедляется